# Import FAVA annotations

In [1]:
import json

with open("/Users/jjr/Documents/data/rarr-rep/input/fava/annotations.json", 'r') as file:
    annotations = json.load(file)

# Create by passage Dataset from annotations

In [2]:
import json
from utilities import create_error_count_dict, extract_gold_answer

overwrite_existing = False

# raar input file will be a list of dictionaries
dataset_bp = [{"input_info":{}} for row in annotations]

# iterate over each row in annotated
for idx, row in enumerate(annotations):
    # -> add id
    dataset_bp[idx]["input_info"]["id"] = idx
    
    # -> add claim
    dataset_bp[idx]["input_info"]["claim"] = annotations[idx]["output"]
    
    # -> add gold
    dataset_bp[idx]["input_info"]["gold"] = extract_gold_answer(annotations[idx]["output"])
    
    # -> add error count dictionary for this passage
    annotated_text = annotations[idx]["annotated"]
    dataset_bp[idx]["input_info"]["error_count_dict"] = create_error_count_dict(annotated_text)
    
    # -> add flag to indicate if sample has an error
    error_counts_dict = dataset_bp[idx]["input_info"]["error_count_dict"]
    # check if any errors for this sample
    has_error = any(value > 0 for value in error_counts_dict.values())
    dataset_bp[idx]['input_info']['has_error'] = has_error

if overwrite_existing:
    # path to the output file
    filename = '/Users/jjr/Documents/data/rarr-rep/input/fava/rarr-input_fava_bp.jsonl'
    # write jsonlines file
    with open(filename, 'w') as f:
        for obj in dataset_bp:
            json_line = json.dumps(obj)  
            f.write(json_line + '\n')     


# Create by Sentence Dataset from passages

## -> helper functions

In [2]:
import re
from collections import defaultdict
from typing import List, Dict, Tuple

def cleanup_lone_annot(sentences):
    cleaned_sentences = []
    buffer = ""
    
    for i, sentence in enumerate(sentences):
        # Match any tag within <> up to 20 characters, alone on a line
        if re.fullmatch(r"<[^>]{1,20}>", sentence.strip()):
            tag = sentence.strip()
            
            if tag.startswith("</"):  # Move to the end of the previous sentence
                if cleaned_sentences:
                    cleaned_sentences[-1] += f" {tag}"
                else:
                    buffer += tag  # If there's no previous sentence, buffer it
            else:  # Move to the beginning of the next sentence
                buffer += tag + " "
                
        else:
            # Append buffer if it exists to the beginning of the current sentence
            sentence = buffer + sentence
            buffer = ""  # Clear buffer after use
            cleaned_sentences.append(sentence)
    
    return cleaned_sentences


def cleanup_leading_end_annot(sentences):
    # Define a regex pattern to match an optional prefix (1-2 chars) and ending tag at the start
    pattern = re.compile(r'^[^a-zA-Z0-9]{0,2}</[^>]+>')

    cleaned_sentences = []
    for i, sentence in enumerate(sentences):
        # Check if the sentence starts with an annotation end or short prefix + annotation end
        match = pattern.match(sentence)
        if match:
            # Extract the annotation and remove it from the current sentence
            annotation = match.group(0)
            sentence = sentence[len(annotation):].strip()
            # Append the annotation to the end of the previous sentence if there is one
            if cleaned_sentences:
                cleaned_sentences[-1] += annotation
        # Add the cleaned or unchanged sentence to the list
        cleaned_sentences.append(sentence)

    return cleaned_sentences

def cleanup_dangling_start_annot(sentences):
    cleaned_sentences = []
    pending_annotation = ""

    # Define regex pattern for matching opening <annotation> at the end of a sentence
    open_annotation_pattern = re.compile(r'(<[^/][^>]{0,19}>)(\S{0,2}\s*\.?)$')
     
    for sentence in sentences:
        # Prepend the pending annotation if present
        if pending_annotation:
            sentence = pending_annotation + " " + sentence
            pending_annotation = ""

        # Check for an opening annotation at the end of the sentence with optional trailing characters
        match = open_annotation_pattern.search(sentence)
        
        if match:
            annotation = match.group(1)
            trailing_chars = match.group(2).strip()
            
            # Remove the annotation and store it for the next sentence
            sentence = sentence[:match.start()] + trailing_chars
            pending_annotation = annotation

        cleaned_sentences.append(sentence.strip())

    

    return [sent for sent in cleaned_sentences if len(sent) > 0]

 


def process_annotations(text: str) -> Tuple[str, Dict[str, int]]:
    # Regex to find annotations and their types
    annotation_pattern = r"<(\w+)>(.*?)</\1>"

    # Dictionary to count annotations
    annotation_counts = defaultdict(int)

    # Function to remove annotation markers but keep the content
    def remove_annotation(match):
        annotation_type = match.group(1)
        content = match.group(2)
        annotation_counts[annotation_type] += 1
        return content  # Return the content without the markers

    # Clean the text by removing annotations
    cleaned_text = re.sub(annotation_pattern, remove_annotation, text)

    # Convert defaultdict to dict for return
    annotation_counts = dict(annotation_counts)

    # # Check if any annotations were found
    # if not annotation_counts:
    #     return text, annotation_counts  # Return the original text if no annotations were found

    return cleaned_text, annotation_counts




def remove_irrelevant_annotations(text: str, keep_errors: List[str])-> str:
    # Regex pattern to match any tag independently, capturing the type
    tag_pattern = re.compile(r'</?([^<>]+)>')

    # Function to replace tags based on the type
    def replace_tags(match):
        # remove '/'
        tag_type = match.group(1).replace('/', '') 
        # Check if the tag type is in the list of errors to keep
        if tag_type in keep_errors:
            return match.group(0)  
        else:
            return '' 

    # Replace tags in the text using the replace_tags function
    return re.sub(tag_pattern, replace_tags, text)



# import re
# from collections import Counter

# def count_error_annotations(text):
#     """
#     Counts the number of error annotations in the given text.

#     Args:
#         text (str): The text containing error annotations.

#     Returns:
#         dict: A dictionary with error types as keys and their counts as values.
#     """
#     # Define the error types you're interested in
#     error_types = ['contradictory', 'relation', 'entity', 'invented', 'unverifiable', 'subjective']
    
#     # Initialize a counter for each error type
#     error_counts = Counter({error_type: 0 for error_type in error_types})

#     # Iterate over each error type and count occurrences
#     for error_type in error_types:
#         # Regex pattern to find tags, case-insensitive
#         pattern = fr'<{error_type}>.*?</{error_type}>'
#         matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
#         error_counts[error_type] += len(matches)
    
#     return dict(error_counts)


def remove_mark_tags(text):
    # This regex pattern matches '<mark>' followed by any content (non-greedy), up to '</mark>'
    pattern = r'<mark>.*?</mark>'
    # The re.DOTALL flag allows '.' to match newline characters as well
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text


## -> Pipeline

In [3]:
annotations[1]['annotated']

'Amdavad ni Gufa is an underground art gallery in Ahmedabad, Gujarat, India. It was designed by the renowned architect Balkrishna Doshi and <subjective><delete>is located on the banks of the Sabarmati River.</delete></subjective>The gallery is known for its unique design, which resembles a cave or a cave-like structure. It is made up of twisted, curved, and uneven shapes that create an immersive and complex experience for visitors.Amdavad ni Gufa is home to various exhibitions and installations that showcase the works of local and international artists. Its aim is to provide a platform for contemporary art and encourage creativity and innovation in the field.<unverifiable><delete>The gallery is a popular tourist attraction and has won several architecture and design awards.</delete></unverifiable> <relation><delete>It is open to the public every day except Monday and admission is free.</delete><mark>it is open to the public every day except monday and public holidays and admission is f

In [4]:
# exceptions:
# -> annotation is tokenized as a sentence -> merge with prev or next
# -> annotation start is at end of a sentence -> merge with next
# -> annotation end is at start of a sentence -> merge with previous
# -> annotation spans multiple sentences (do nothing at this point)
# -> two sentences tokenized as one sentence
# other -> [21]

In [5]:
annotations[0].keys()

dict_keys(['prompt', 'output', 'annotated', 'subject', 'dataset', 'model'])

In [6]:
import spacy

error_types = ['contradictory', 'relation', 'entity', 'invented', 'unverifiable', 'subjective']

spacy_nlp = spacy.load("en_core_web_lg")


processed_sentences = []
concat_passages = []
id = 0
for i_samp, sample in enumerate(annotations):
    # if i_samp != 26:
    #     continue
    annot_claim = sample['annotated']
    passage = sample['output']

    # remove all <mark> </mark> tags and the content between the tags
    unmarked_claim = remove_mark_tags(annot_claim)
    
    # preprocess claim - remove all non-error annotations
    # print(annot_claim, '\n')
    clean_claim = remove_irrelevant_annotations(unmarked_claim, error_types)
    
    # perform sentence tokenization on clean claim
    spacy_claim = spacy_nlp(clean_claim)
    sentences = [sent.text for sent in spacy_claim.sents]

    # cleanup edge cases
    sentences = cleanup_lone_annot(sentences)
    sentences = cleanup_leading_end_annot(sentences)
    sentences = cleanup_dangling_start_annot(sentences)
    
    # iterate over sentences and extract errors
    concat_passage = ""
    for i_sent, sentence in enumerate(sentences):
        # don't process sentences if 3 chars or less
        if len(sentence) <= 3:
            concat_passage += sentence
            continue
            
        data_dict = {"annotated_sentence": sentence}
        data_dict['annotated_passage'] = annot_claim
        data_dict['id'] = id
        data_dict['pid'] = i_samp
        data_dict['sid'] = i_sent
        # data_dict['passage'] = passage
        # print(f"sentence {i_sent}: {sentence}\n")
        # process sentence
        processed_sent, error_count_dict = process_annotations(sentence)
        # if no errors and processed
        # remove any remaining tags
        processed_sent = re.sub(r'</?[^>]+>', '', processed_sent)
        if len(concat_passage) == 0:
            concat_passage += processed_sent
        else:
            concat_passage = concat_passage + " " + processed_sent
        data_dict['sentence'] = processed_sent
        data_dict['error_count_dict'] = error_count_dict
        # print(f"processed sentence: {i_sent}: {processed_sent}\n")
        processed_sentences.append({"input_info":data_dict})
        
        if error_count_dict:
            data_dict['has_error'] = True
        else:
            data_dict['has_error'] = False
        id+=1
        
    concat_passages.append({'pid': i_samp, "concat_passage": concat_passage })
    

# now add concatenated passage to each sentence data dictionary
for i, obj in enumerate(processed_sentences):
    pid = obj['input_info']['pid']
    concat_passage = concat_passages[pid]["concat_passage"]
    obj['input_info']['concat_passage'] = concat_passage

# Write to a .jsonl file
with open('./data/rarr-input_fava_bs_all_rev2.jsonl', 'w') as f:
    for entry in processed_sentences:
        f.write(json.dumps(entry) + '\n')

In [12]:
processed_sentences[0].keys()

dict_keys(['input_info'])

In [7]:
processed_sentences[1000]['input_info'].keys()

dict_keys(['annotated_sentence', 'annotated_passage', 'id', 'pid', 'sid', 'sentence', 'error_count_dict', 'has_error', 'concat_passage'])

In [22]:
annotations[9]

{'prompt': 'What are the Golden Lion movies at the Venice Film Festival whose production countries are in Asia?',
 'output': 'The Golden Lion is the highest prize awarded at the Venice Film Festival, which is one of the most prestigious film festivals in the world. The festival has a long history of showcasing and rewarding films from diverse countries and cultures. Here are some Golden Lion movies at the Venice Film Festival whose production countries are in Asia:1. "The Battle of Algiers" (1966) - Directed by Gillo Pontecorvo, this film is a powerful depiction of the Algerian struggle for independence from France. The film was produced in Algeria, a country in North Africa, but it has been recognized as a classic of world cinema and has had a significant impact on filmmakers across the globe.2. "Theriority of the Family" (1974) - Directed by Ousmane Sembene, this film is a powerful critique of the traditional family structure in Senegal and the impact of colonialism on African societ

In [ ]:
# Write to a .jsonl file
with open('./data/rarr-input_fava_bs_all_rev2.jsonl', 'w') as f:
    for entry in processed_sentences:
        f.write(json.dumps(entry) + '\n')

In [31]:

with open("/Users/jjr/Documents/data/rarr-rep/fava/annotations.json", 'r') as file:
    annotations = json.load(file)

In [11]:
num_mark = 0
mark_ids = []
for i,sample in enumerate(annotations):
    annot_claim = sample['annotated']
    if "<mark>" in annot_claim:
        num_mark += 1
        mark_ids.append(i)

In [12]:
mark_ids[1]

3

In [13]:
annotations[3]['annotated']

'A Long Long Way is a novel written by Sebastian Barry. It was first published in 2005 by <entity><delete>Faber and Faber</delete><mark>\tviking press</mark></entity>, a <entity><delete>British</delete><mark>american</mark></entity> publishing company. The novel tells the story of Willie Dunne, an Irish soldier who has a strong sense of loyalty to his country, but also feels torn between his Irishness and his loyalty to the British army during World War I. The novel is set against the backdrop of the Easter Rising in Dublin in 1916, and the subsequent Irish War of Independence. It explores themes of nationalism, identity, family, and sacrifice. The novel was critically acclaimed and received several literary awards, <unverifiable><delete>including the 2005 Dublin Literary Award</delete></unverifiable> and was shortlisted for the Man Booker Prize in the same year.'

In [24]:
for sample in fava_all:
    pas

SyntaxError: incomplete input (2712574183.py, line 1)

In [234]:
# # Write to a .jsonl file
# with open('./data/rarr-input_fava_bs.jsonl', 'w') as f:
#     for entry in processed_sentences:
#         f.write(json.dumps(entry) + '\n')

# Analyze BS Dataset

In [7]:
fava_bs_all = []
with open('./data/rarr-input_fava_bs_all_rev2.jsonl', 'r') as f:
    for line in f:
        fava_bs_all.append(json.loads(line))

In [9]:
len(fava_bs_all)

5150

In [20]:
fava_bs_all[24]['input_info']

{'annotated_sentence': 'It was first published in 2005 by <entity>Faber and Faber</entity>, a <entity>British</entity> publishing company.',
 'annotated_passage': 'A Long Long Way is a novel written by Sebastian Barry. It was first published in 2005 by <entity><delete>Faber and Faber</delete><mark>\tviking press</mark></entity>, a <entity><delete>British</delete><mark>american</mark></entity> publishing company. The novel tells the story of Willie Dunne, an Irish soldier who has a strong sense of loyalty to his country, but also feels torn between his Irishness and his loyalty to the British army during World War I. The novel is set against the backdrop of the Easter Rising in Dublin in 1916, and the subsequent Irish War of Independence. It explores themes of nationalism, identity, family, and sacrifice. The novel was critically acclaimed and received several literary awards, <unverifiable><delete>including the 2005 Dublin Literary Award</delete></unverifiable> and was shortlisted for 

In [9]:
# collect the split sentences
sentences_all = []

for sample in fava_bs_all:
    sentence = sample['input_info']['sentence']
    annot_sentence = sample['input_info']['annotated_sentence']
    sentences_all.append({'as':annot_sentence, 's':sentence})




In [10]:
fava_bs_all[0]['input_info'].keys()

dict_keys(['annotated_sentence', 'annotated_passage', 'id', 'pid', 'sid', 'sentence', 'error_count_dict', 'has_error', 'concat_passage'])

In [11]:
len(sentences_all)

5185

In [12]:
short_sents

NameError: name 'short_sents' is not defined

In [55]:
# collect the split sentences
passages_all = [{}]

for sample in fava_bs_all:
    passage = sample['input_info']['concat_passage']
    passages_all.append(passage)

In [57]:
for p in passages_all:
    if "*" in p:
        print(p)

In [63]:
short_sents

140

In [19]:
short_sents = 0
for sentence in sentences_all:
    sent = sentence['s']
    annot_sent = sentence['as']
    if len(sent) <= 5:
        short_sents += 1
        print(f"annot sent: {annot_sent}")
        print(f"sentence: {sent}")       

annot sent: 1634:
sentence: 1634:
annot sent: 1634:
sentence: 1634:
annot sent: <invented>"The
sentence: "The
annot sent: Sure!
sentence: Sure!
annot sent: 1634:
sentence: 1634:
annot sent: Sure!
sentence: Sure!
annot sent: 1634:
sentence: 1634:
annot sent: Flag:
sentence: Flag:
annot sent: <entity> *
sentence:  *
annot sent: Sure!
sentence: Sure!
annot sent: 2010.</entity>
sentence: 2010.


In [19]:
fava_bs_all[0]['input_info'].keys()

dict_keys(['annotated_sentence', 'annotated_passage', 'id', 'cid', 'sid', 'passage', 'sentence', 'error_count_dict', 'has_error'])

In [248]:
fava_bs = []
with open('./data/rarr-input_fava_bs.jsonl', 'r') as f:
    for line in f:
        fava_bs.append(json.loads(line))

# number of sentences
num_sentences = len(fava_bs)
error_count_total_dict = {key:0 for key in error_types}
num_errors_total = 0
for sample in fava_bs:
    error_count_dict = sample['input_info']['error_count_dict']
    num_errors = sum(error_count_dict.values())
    num_errors_total+=num_errors
    for error_type, count in error_count_dict.items():
        error_count_total_dict[error_type] += count
# number of sentences with > 1 error
# number of errors by type



In [249]:
error_count_total_dict

{'contradictory': 159,
 'relation': 24,
 'entity': 460,
 'invented': 98,
 'unverifiable': 130,
 'subjective': 125}

In [251]:
passage_error_count_total_dict = {'entity': 487,
                                  'contradictory': 174,
                                  'unverifiable': 151,
                                  'invented': 150,
                                  'subjective': 138,
                                  'relation': 32}

In [258]:
print(f"number of sentences in FAVA BS: {num_sentences}")
print(f"number of errors in FAVA BS: {num_errors_total}")
print(f"number of errors in FAVA BP: {1132}")
print(f"number of stranded errors in FAVA BS: {1132-num_errors_total}")
print("errors by error type:")
for error_type, count in error_count_total_dict.items():
    print(f"\t{error_type}: {count}")

number of sentences in FAVA BS: 5256
number of errors in FAVA BS: 996
number of errors in FAVA BP: 1132
number of stranded errors in FAVA BS: 136
errors by error type:
	contradictory: 159
	relation: 24
	entity: 460
	invented: 98
	unverifiable: 130
	subjective: 125


# BS Stratified Sample

In [318]:
# # convert to dataframe
# import pandas as pd

# # Define error mapping
# error_mapping = {
#     'contradictory': 'e1',
#     'relation': 'e2',
#     'entity': 'e3',
#     'invented': 'e4',
#     'unverifiable': 'e5',
#     'subjective': 'e6'
# }

# # Create a list to hold the DataFrame rows
# rows = []

# # Loop through each item in the dataset
# for item in processed_sentences:
#     row_info = item['input_info']
#     row = {
#         'id': row_info['id'],
#         'e1': row_info['error_count_dict'].get('contradictory', 0),
#         'e2': row_info['error_count_dict'].get('relation', 0),
#         'e3': row_info['error_count_dict'].get('entity', 0),
#         'e4': row_info['error_count_dict'].get('invented', 0),
#         'e5': row_info['error_count_dict'].get('unverifiable', 0),
#         'e6': row_info['error_count_dict'].get('subjective', 0),
#     }
#     rows.append(row)

# # Create DataFrame
# df = pd.DataFrame(rows, columns=['id', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6'])



In [343]:
# # convert to binary
# df_binary = df.copy()
# df_binary.loc[:, 'e1':'e6'] = (df_binary.loc[:, 'e1':'e6'] >= 1).astype(int)

# for col in ['e2', 'e4', 'e6', 'e5', 'e1', 'e3']:
#     print(col, 'method 1:', round(df_binary[col].mean(), 4))
#     print(col, 'method 2:', df_binary[df_binary[col] == 1][col].sum()/len(df_binary), '\n')

e2 method 1: 0.0046
e2 method 2: 0.0045662100456621 

e4 method 1: 0.0164
e4 method 2: 0.016362252663622526 

e6 method 1: 0.0221
e6 method 2: 0.02207001522070015 

e5 method 1: 0.0219
e5 method 2: 0.021879756468797563 

e1 method 1: 0.0263
e1 method 2: 0.026255707762557076 

e3 method 1: 0.0712
e3 method 2: 0.07115677321156773 



In [344]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedShuffleSplit

# # Desired sample size
# sample_size = 460

# # Initialize DataFrame to store the sampled indices
# sampled_indices = pd.Index([])

# sample_fraction = sample_size / len(df_binary)

# # Stratified sampling for each error type
# for error in ['e2', 'e4', 'e6', 'e5', 'e1', 'e3']:
#     splitter = StratifiedShuffleSplit(n_splits=1, test_size=sample_fraction, random_state=42)
#     for _, test_index in splitter.split(df_binary, df_binary[error]):
#         sampled_indices = sampled_indices.union(test_index)

# # Deduplicate indices in case some rows are selected multiple times
# sampled_indices = sampled_indices.unique()

# # If we have more samples than needed, randomly drop the excess
# if len(sampled_indices) > sample_size:
#     sampled_indices = np.random.choice(sampled_indices, size=sample_size, replace=False)

# # Get the stratified sample DataFrame
# stratified_sample = df_binary.loc[sampled_indices]



In [346]:
# for col in ['e1', 'e2', 'e3', 'e4', 'e5', 'e6']:
#     print(col, 'method 1:', round(stratified_sample[col].mean(), 4))

e2 method 1: 0.0022
e4 method 1: 0.0217
e6 method 1: 0.0196
e5 method 1: 0.0196
e1 method 1: 0.0261
e3 method 1: 0.063


In [362]:
# stratified_sample_ids = stratified_sample['id'].tolist()
# stratified_sample_ids = sorted(stratified_sample_ids)
# processed_sentences_sample = [processed_sentences[i] for i in stratified_sample_ids]

In [364]:
# # Write to a .jsonl file
# with open('./data/rarr-input_fava_bs.jsonl', 'w') as f:
#     for entry in processed_sentences_sample:
#         f.write(json.dumps(entry) + '\n')

In [367]:
# # Write to a .jsonl file
# with open('./data/rarr-input_fava_bs_all.jsonl', 'w') as f:
#     for entry in processed_sentences:
#         f.write(json.dumps(entry) + '\n')

In [366]:
# len(processed_sentences)

5256

In [375]:
# processed_sentences_errors = [sample for sample in processed_sentences if sample['input_info']['has_error']]

In [377]:
# # Write to a .jsonl file
# with open('./data/rarr-input_fava_bs_all_errors.jsonl', 'w') as f:
#     for entry in processed_sentences_errors:
#         f.write(json.dumps(entry) + '\n')

In [371]:
# # processed_sentences -> errors only
# mask = (df_binary.e1 == 1) | (df_binary.e2 == 1) | (df_binary.e3 == 1) | (df_binary.e4 == 1) | (df_binary.e5 == 1) | (df_binary.e6 == 1)
# df_binary[mask]

,id,e1,e2,e3,e4,e5,e6
6,6,0,0,0,0,0,1
10,10,0,0,0,0,1,0
11,11,0,1,0,0,0,0
24,24,0,0,1,0,0,0
28,28,0,0,0,0,1,0
...,...,...,...,...,...,...,...
5176,5176,0,0,1,0,0,0
5178,5178,0,0,1,0,0,0
5179,5179,0,0,1,0,0,0
5180,5180,0,0,1,0,0,0


## -> helper function

In [8]:
from typing import List, Dict

def display_rarr_output(idx_start: int, 
                        idx_end:int,
                       annotations: List[Dict[str,any]],
                       output: List[Dict[str,any]],
                        display_filter: List[int]=None,
                       show_revision_steps: bool = True):
    for idx in range(idx_start, idx_end, 1):
    # for idx in range(0,224):
        
        if display_filter and idx not in display_filter:
            continue
        print(f"ID:{idx}\n")
        show_revision_steps = True
        
        annotated_gold = annotations[idx]["annotated"]
        
        # annotated_rarr = output_8b[idx]["result"]["annotations"]
        
        claim = output[idx]["result"]["claim"]
        print(f"--------------CLAIM--------------\n{claim}\n")
        
        try:
            questions = output[idx]["result"]["agreement_gates"]
            if questions is None: questions = []
        except KeyError:
            questions = []
        print(f"--------------QUESTIONS--------------\nNo. of questions generated: {len(questions)}\n")
        
        try:
            if show_revision_steps:
                print("--------------REVISIONS--------------")
                agreement_gates = output[idx]["result"]["agreement_gates"]
                no_revisions = True
                for agr,rev in zip(agreement_gates, output[idx]["result"]["revisions"]):
                    gate_status = agr["is_open"]
                    if gate_status:
                        no_revisions = False
                        reason = agr["reason"]
                        print(f"-->Edit Gate Open?: {gate_status}\n-->reason: {reason}\n")
                        print(rev['revised_claim'], "\n")
                if no_revisions:
                    print("No Revisions")
        except (TypeError, KeyError) as e:
            print("\n", "Malfunction - no questions", "\n")
        
        revised_claim = output[idx]["result"]["revised_claim"]
        print(f"--------------REVISED CLAIM--------------:\n{revised_claim}\n")
        
        # print("--------------ERROR ANNOTATIONS--------------")
        # print(f"gold:{results[idx]['gold']}\n")
        # print(f"rarr:{results[idx]['rarr']}\n")
        # print(f"****** gold annotated ******:\n{annotated_gold}\n\n****** rarr annotated ******:\n{annotated_rarr}\n")
        print("********************************************************************************************************")
        print("********************************************************************************************************")
        print("********************************************************************************************************")
